In [3]:
import sys
from pathlib import Path
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
sys.path.insert(0, str(project_root))

import numpy as np
from sklearn.model_selection import train_test_split
from src.neural_network import NeuralNetwork
from src.activations import Sigmoid, Linear
from src.losses import MSE
from src.optimizers import GD, RMSprop, Adam
from src.training import train
from src.metrics import mse
from src.utils import runge, polynomial_features, scale_data, OLS_parameters, inverse_scale_y

import torch
import torch.nn as torch_nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from autograd import grad
import autograd.numpy as anp

# Setup for NN

In [4]:
SEED = 42
np.random.seed(SEED)

N = 300
x = np.linspace(-1, 1, N)
y_true = runge(x)
y_noise = y_true + np.random.normal(0, 0.01, N)

In [5]:
X_train_raw, X_test_raw, y_train_nn, y_test_nn = train_test_split(
    x.reshape(-1, 1), y_noise.reshape(-1, 1), 
    test_size=0.2, random_state=SEED
)

# Scale
X_train_s, y_train_s, X_mean, X_std, y_mean = scale_data(X_train_raw, y_train_nn)
X_test_s, y_test_s, _, _, _ = scale_data(X_test_raw, y_test_nn, X_mean, X_std, y_mean)

y_test_real = inverse_scale_y(y_test_s, y_mean)

In [6]:
network_input_size = 1
layer_output_sizes = [100, 100, 1]  # Two hidden layers with 100 nodes each
activations = [Sigmoid(), Sigmoid(), Linear()]
loss = MSE()
epochs = 1000
batch_size = 32
eta = 0.001

nn = NeuralNetwork(network_input_size=network_input_size, layer_output_sizes=layer_output_sizes, activations=activations, loss=loss, seed=SEED)
train(nn, X_train_s, y_train_s, X_test_s, y_test_s, Adam(eta), 
      epochs=epochs, batch_size=batch_size, verbose=True, seed=SEED) 
y_pred_s = nn.predict(X_test_s)
y_pred = inverse_scale_y(y_pred_s, y_mean)
nn_mse = mse(y_test_real, y_pred)
print(f"Neural Network MSE on test set: {nn_mse}")

[SGD] Epoch   0/1000 - Train Loss: 0.0867, Val Loss: 0.0717, Val MSE: 0.0717
[SGD] Epoch  10/1000 - Train Loss: 0.0837, Val Loss: 0.0684, Val MSE: 0.0684
Early stopping at epoch 14 (patience=10)
Neural Network MSE on test set: 0.0682396050584541


# Setup for NN with Pytorch

In [7]:
torch.manual_seed(SEED)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_s).to(device)
y_train_tensor = torch.FloatTensor(y_train_s).to(device)
X_test_tensor = torch.FloatTensor(X_test_s).to(device)
y_test_tensor = torch.FloatTensor(y_test_s).to(device)

# Create DataLoader for mini-batch training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define the neural network
class RungeNet(torch_nn.Module):  # ← Use torch_nn
    def __init__(self):
        super(RungeNet, self).__init__()
        self.fc1 = torch_nn.Linear(1, 100)      # ← Use torch_nn
        self.fc2 = torch_nn.Linear(100, 100)    # ← Use torch_nn
        self.fc3 = torch_nn.Linear(100, 1)      # ← Use torch_nn
        self.sigmoid = torch_nn.Sigmoid()       # ← Use torch_nn
        
    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = RungeNet().to(device)

# Loss function and optimizer
criterion = torch_nn.MSELoss()  # ← Use torch_nn
optimizer = optim.Adam(model.parameters(), lr=0.001)


epochs = 1000
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    # Print progress every 100 epochs
    if (epoch + 1) % 100 == 0:
        avg_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.6f}")

# Evaluation on test set
model.eval()
with torch.no_grad():
    y_pred_pytorch_s = model(X_test_tensor).cpu().numpy()

# Inverse scale predictions (same as your code)
y_pred_pytorch = inverse_scale_y(y_pred_pytorch_s, y_mean)

# Compute MSE (same as your code)
pytorch_mse = mse(y_test_real, y_pred_pytorch)

print(f"PyTorch MSE: {pytorch_mse:.6f}")

Epoch 100/1000, Loss: 0.027220
Epoch 200/1000, Loss: 0.020118
Epoch 300/1000, Loss: 0.006624
Epoch 400/1000, Loss: 0.000560
Epoch 500/1000, Loss: 0.000174
Epoch 600/1000, Loss: 0.000123
Epoch 700/1000, Loss: 0.000109
Epoch 800/1000, Loss: 0.000133
Epoch 900/1000, Loss: 0.000111
Epoch 1000/1000, Loss: 0.000125
PyTorch MSE: 0.000080


# Comparing our model with PyTorch

In [8]:
print(f"\n{'='*60}")
print(f"Our Neural Network MSE: {nn_mse:.6f}")
print(f"PyTorch MSE:             {pytorch_mse:.6f}")
print(f"Difference:              {abs(pytorch_mse - nn_mse):.6f}")
print(f"{'='*60}")


Our Neural Network MSE: 0.068240
PyTorch MSE:             0.000080
Difference:              0.068160


# Verifying derivatives with Autograd

### MSE gradient

In [9]:
y_true = np.array([[1.0], [2.0], [3.0]])
y_pred = np.array([[1.1], [1.9], [3.2]])

loss = MSE()
our_loss = loss.forward(y_true, y_pred)
our_grad = loss.backward(y_true, y_pred)

def mse_auto(y_pred, y_true): 
    return anp.mean((y_true - y_pred)**2)

auto_grad_fn = grad(mse_auto, 0)
auto_grad = auto_grad_fn(y_pred, y_true)

print(f'Our gradient:      {our_grad.flatten()}')
print(f'Autograd gradient: {auto_grad.flatten()}')

Our gradient:      [ 0.06666667 -0.06666667  0.13333333]
Autograd gradient: [ 0.06666667 -0.06666667  0.13333333]


### Backpropagation

In [10]:
test_nn = NeuralNetwork(
    network_input_size=2, 
    layer_output_sizes=[3, 1],
    activations=[Sigmoid(), Linear()], 
    loss=MSE(), 
    seed=SEED
)

X_test = np.array([[0.5, 0.3], [0.2, 0.8]])
y_test = np.array([[1.0], [0.5]])

# Compute gradients with backpropagation
our_grads = test_nn.compute_gradient(X_test, y_test)

# Define forward pass for autograd
def nn_loss_auto(layers, X, y, activations):
    a = X
    for i, ((W, b), act) in enumerate(zip(layers, activations)):
        z = anp.dot(a, W.T) + b
        # Apply activation
        if isinstance(act, Sigmoid):
            a = 1 / (1 + anp.exp(-z))
        else:  # Linear
            a = z
    return anp.mean((y - a)**2)

# Compute gradients with autograd
auto_grad_fn = grad(nn_loss_auto, 0)
auto_grads = auto_grad_fn(test_nn.layers, X_test, y_test, test_nn.activations)

# Compare gradients for each layer
print(f"Network: {X_test.shape[1]} -> 3 -> 1")
print(f"Number of layers: {len(our_grads)}")

all_correct = True
for layer_idx in range(len(our_grads)):
    W_ours, b_ours = our_grads[layer_idx]
    W_auto, b_auto = auto_grads[layer_idx]
    
    w_diff = np.max(np.abs(W_ours - W_auto))
    b_diff = np.max(np.abs(b_ours - b_auto))
    
    layer_correct = (w_diff < 1e-6 and b_diff < 1e-6)
    all_correct = all_correct and layer_correct
    
    print(f"\nLayer {layer_idx}:")
    print(f"  Weight gradient max diff: {w_diff:.2e}")
    print(f"  Bias gradient max diff:   {b_diff:.2e}")
    print(f"  ✓ Layer {layer_idx} correct: {layer_correct}")

print(f"\n{'='*60}")
print(f"✓ ALL GRADIENTS CORRECT: {all_correct}")
print(f"{'='*60}")

Network: 2 -> 3 -> 1
Number of layers: 2

Layer 0:
  Weight gradient max diff: 3.47e-18
  Bias gradient max diff:   6.94e-18
  ✓ Layer 0 correct: True

Layer 1:
  Weight gradient max diff: 0.00e+00
  Bias gradient max diff:   0.00e+00
  ✓ Layer 1 correct: True

✓ ALL GRADIENTS CORRECT: True
